# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [2]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [3]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

There might be a problem with your API key? Please visit the troubleshooting notebook!


In [4]:
!ollama pull llama3.2

from openai import OpenAI
MODEL = "llama3.2"
openai = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

response = openai.chat.completions.create(
 model=MODEL,
 messages=[{"role": "user", "content": "What is 2 + 2?"}]
)

print(response.choices[0].message.content)

# If this doesn't work, try Kernel menu >> Restart Kernel and Clear Outputs Of All Cells, then run the cells from the top of this notebook down.
# If it STILL doesn't work (horrors!) then please see the Troubleshooting notebook in this folder for full instructions

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 
2 + 2 = 4


In [5]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [6]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [7]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [8]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [9]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [10]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [11]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [12]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/deepseek-ai/DeepSeek-R1-0528',
 '/ResembleAI/chatterbox',
 '/deepseek-ai/DeepSeek-R1-0528-Qwen3-8B',
 '/google/gemma-3n-E4B-it-litert-preview',
 '/osmosis-ai/Osmosis-Structure-0.6B',
 '/models',
 '/spaces/ResembleAI/Chatterbox',
 '/spaces/enzostvs/deepsite',
 '/spaces/multimodalart/wan2-1-fast',
 '/spaces/wushuang98/Direct3D-S2-v1.0-demo',
 '/spaces/alexnasa/Chain-of-Zoom',
 '/spaces',
 '/datasets/yandex/yambda',
 '/datasets/open-r1/Mixture-of-Thoughts',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/MiniMaxAI/SynLogic',
 '/datasets/Hcompany/WebClick',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs

In [14]:
get_links("https://huggingface.co")

{'links': [{'type': 'About page', 'url': 'https://huggingface.co'},
  {'type': 'Company page', 'url': 'https://huggingface.co/brand'},
  {'type': 'Changelog', 'url': 'https://huggingface.co/changelog'},
  {'type': 'Blog', 'url': 'https://blog.huggingface.co'}]}

In [27]:
get_links("https://www.airlock.com")

{'links': [{'type': 'about page', 'url': 'https://airlock.com/en/about-us'},
  {'type': 'company page', 'url': 'https://airlock.com/'},
  {'type': 'careers/jobs page', 'url': 'https://airlock.com/team/jobs'}]}

In [29]:
get_links("https://www.ergon.ch")

{'links': [{'type': 'About page', 'url': '/de/ueber-ergon'},
  {'type': 'Careers/Jobs page', 'url': '/de/karriere/jobs'},
  {'type': 'Company Overview page', 'url': '/de/ueber-ergon/unternehmen'},
  {'type': 'News Archive page', 'url': '/de/news?category=vor%20jahren'},
  {'type': 'Branch Networks page', 'url': '/de/branchen'},
  {'type': 'Services page', 'url': '/de/services'},
  {'type': 'Contact us page', 'url': '/de/kontakt'},
  {'type': 'Social Media links',
   'url': ['https://www.linkedin.com/company/ergon-informatik-ag/',
    'https://www.xing.com/companies/ergoninformatikag',
    'https://www.youtube.com/@ErgonInformatik',
    'http://www.facebook.com/ErgonAG',
    'https://twitter.com/ErgonAG',
    'https://www.instagram.com/ergoninformatik/']},
  {'type': 'Media Corner', 'url': '/de/media-corner'},
  {'type': 'Impressum page', 'url': '/de/impressum'}]}

In [30]:
get_links("http://www.airlock.com/microgateway")

{'links': [{'type': 'company page', 'url': 'https://www.airlock.com/'},
  {'type': 'About Us',
   'url': 'https://techzone.ergon.ch/en/secure-access-hub/components/microgateway?gad_campaignid=22637043751&cHash=19325fb08aab86dc021d9f7785d08e76'},
  {'type': 'Features',
   'url': 'https://airlock.com/secure-access-hub/features/in-the-cloud'},
  {'type': 'Products',
   'url': 'https://airlock.com/secure-access-hub/components/microgateway'},
  {'type': 'Blog', 'url': 'https://github.com/airlock'},
  {'type': 'Support', 'url': 'https://airlock.com/support/'}]}

In [31]:
get_links("https://www.airlock.com/en/secure-access-hub/components/microgateway")

{'links': [{'type': 'Company Overview', 'url': 'https://www.airlock.com/en/'},
  {'type': 'Whitepaper Overview',
   'url': 'https://techzone.ergon.ch/secure-access-hub/en/whitepaper-overview'},
  {'type': 'Glossary',
   'url': 'https://techzone.ergon.ch/secure-access-hub/en/glossary'},
  {'type': 'Contact', 'url': 'https://techzone.ergon.ch/en/contact'},
  {'type': 'About Ergon', 'url': 'https://en/about-ergon/airlock'}]}

In [28]:
get_links("https://docs.airlock.com/microgateway/")

{'links': [{'type': 'company page',
   'url': 'https://docs.airlock.com/microgateway/'},
  {'type': 'about webpage on MicroGateway',
   'url': 'https://docs.airlock.com/microgateway/about'}]}

In [32]:
get_links("https://gateway.envoyproxy.io/")

{'links': [{'type': 'About page',
   'url': 'https://gateway.envoyproxy.io/about'},
  {'type': 'Company page', 'url': 'https://www.tetrate.io'},
  {'type': 'Contribution page',
   'url': 'https://contributions envoyproxy org/'},
  {'type': 'GitHub page', 'url': 'https://github.com/envoyproxy/gateway'}]}

In [33]:
get_links("https://gateway.envoyproxy.io/docs/")

{'links': [{'type': 'About page', 'url': 'https://gateway.envoyproxy.io/'},
  {'type': 'Company website', 'url': 'https://www.envoyproxy.io'},
  {'type': 'GitHub repository',
   'url': 'https://github.com/envoyproxy/gateway'},
  {'type': 'LinkedIn company page',
   'url': 'https://www.linkedin.com/company/envoy-cloud-native'},
  {'type': 'Community invitation page',
   'url': 'https://communityinviter.com/apps/envoyproxy/envoy'}]}

In [34]:
get_links("https://gateway.envoyproxy.io/docs/tasks/security/oidc/")

{'links': [{'type': 'official website', 'url': 'https://envoyproxy.io'},
  {'type': 'blog/news', 'url': 'https://envoymobile.io'},
  {'type': 'LinkedIn company page',
   'url': 'https://www.linkedin.com/company/envoy-cloud-native'},
  {'type': 'GitHub repository',
   'url': 'https://github.com/envoyproxy/gateway'}]}

In [35]:
get_links("https://kgateway.dev/")

{'links': [{'type': 'about page', 'url': 'https://kgateway.dev/docs/about/'},
  {'type': 'company information', 'url': 'https://www.solo.io/kgatewaydev'},
  {'type': 'LinkedIn company profile',
   'url': 'https://www.linkedin.com/company/kgateway/'},
  {'type': 'GitHub repository for kgateway',
   'url': 'https://github.com/kgateway-dev/kgateway'},
  {'type': 'mastodon social media handle',
   'url': 'https://mastodon.social/@kgateway'},
  {'type': 'YouTube channel for kgateway',
   'url': 'https://youtube.com/@kgateway-dev'}]}

In [36]:
get_links("https://kgateway.dev/docs/")

{'links': [{'type': 'Company page',
   'url': 'https://github.com/kgateway-dev/kgateway'},
  {'type': 'Setup guide', 'url': 'https://docs.solo.io/'},
  {'type': 'Blog', 'url': 'https://blog.solo.io/'},
  {'type': 'Learn more', 'url': 'https://learn.solo.io/'},
  {'type': 'Slack channel', 'url': 'https://slack.com/kgatewaydev'},
  {'type': 'Social media profiles',
   'url': ['https://bsky.app/profile/kgateway.dev',
    'https://mastodon.social/@kgateway',
    'https://www.linkedin.com/company/kgateway/',
    'https://youtube.com/@kgateway-dev']},
  {'type': 'Documentation', 'url': 'https://docs.solo.io/api/'}]}

In [37]:
get_links("https://kgateway.dev/docs/mcp/")

{'links': [{'type': 'about page',
   'url': 'https://modelcontextprotocol.io/introduction'},
  {'type': 'contact email', 'url': 'http://127.0.0.1:6274'},
  {'type': 'Company website', 'url': 'https://kgateway.dev/docs/mcp/'},
  {'type': 'Documentation',
   'url': 'https://modelcontextprotocol.io specification/2025-03-26/basic/authorization/'},
  {'type': 'Blog', 'url': 'https://www.solo.io/'},
  {'type': 'Social media channels',
   'url': ['https://x.com/kgatewaydev',
    'https://bsky.app/profile/kgateway.dev',
    'https://mastodon.social/@kgateway']},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/kgateway/'},
  {'type': 'YouTube channel', 'url': 'https://youtube.com/@kgateway-dev'},
  {'type': 'GitHub repository',
   'url': 'https://github.com/kgateway-dev/kgateway'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [15]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [39]:
print(get_all_details("https://gateway.envoyproxy.io/"))

Found links: {'links': [{'type': 'about page', 'url': 'https://gateway.envoyproxy.io/docs'}, {'type': 'Contributions page', 'url': 'https://gateway.envoyproxy.io/contributions'}, {'type': 'FAQs or About page', 'url': 'https://aigateway.envoyproxy.io'}]}
Landing page:
Webpage Title:
Envoy Gateway
Webpage Contents:
Envoy Gateway
Documentation
News
About
Contributions
Versions
latest
v1.4
v1.3
v1.2 (EOL)
v1.1 (EOL)
v1.0 (EOL)
v0.6 (EOL)
v0.5 (EOL)
v0.4 (EOL)
v0.3 (EOL)
v0.2 (EOL)
Envoy Gateway
Manage your Application and API traffic with Envoy Gateway.
Aimed at making it easy to adopt, use, and manage Envoy Proxy.
Deploy as a Standalone or Kubernetes-based API Gateway, implementing and extending the Kubernetes Gateway API.
Get Started
GitHub
Built together. Built in the open.
Gateway is the result of the community coming together to make it easier than ever to leverage Envoy Proxy for your API Gateway needs.
Get involved in the community
Join our community on Slack, join the conversation 

In [40]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [41]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [42]:
get_brochure_user_prompt("Envoy Gateway", "https://gateway.envoyproxy.io/")

Found links: {'links': [{'type': 'About page', 'url': 'https://gateway.envoyproxy.io/about'}, {'type': 'Contribution page', 'url': 'https://gateway.envoyproxy.io/contributions'}, {'type': 'Community link', 'url': 'https://communityinviter.com/apps/envoyproxy/envoy'}]}


"You are looking at a company called: Envoy Gateway\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nEnvoy Gateway\nWebpage Contents:\nEnvoy Gateway\nDocumentation\nNews\nAbout\nContributions\nVersions\nlatest\nv1.4\nv1.3\nv1.2 (EOL)\nv1.1 (EOL)\nv1.0 (EOL)\nv0.6 (EOL)\nv0.5 (EOL)\nv0.4 (EOL)\nv0.3 (EOL)\nv0.2 (EOL)\nEnvoy Gateway\nManage your Application and API traffic with Envoy Gateway.\nAimed at making it easy to adopt, use, and manage Envoy Proxy.\nDeploy as a Standalone or Kubernetes-based API Gateway, implementing and extending the Kubernetes Gateway API.\nGet Started\nGitHub\nBuilt together. Built in the open.\nGateway is the result of the community coming together to make it easier than ever to leverage Envoy Proxy for your API Gateway needs.\nGet involved in the community\nJoin our community on Slack, join the conversation on GitHub, and attend our c

In [21]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [43]:
create_brochure("Envoy Gateway", "https://gateway.envoyproxy.io/")

Found links: {'links': [{'type': 'about page', 'url': 'https://gateway.envoyproxy.io/about'}, {'type': 'contributions page', 'url': 'https://gateway.envoyproxy.io/contributions'}]}


# Envoy Gateway Brochure
## Introduction

Envoy Gateway is an open-source project that aims to make it easier to adopt, use, and manage Envoy Proxy as a Kubernetes Gateway. Deployment can be done as a standalone or Kubernetes-based API gateway, implementing and extending the Kubernetes Gateway API.

## Company Culture

At Envoy Gateway, we are committed to building something great together. Our community-driven approach ensures that our product is always evolving to meet the needs of users like you.

* **Built for Collaboration**: We believe that collaboration is key to greatness. That's why we're built on an open-source model and actively encourage contributors from around the world.
* **Transparency and Communication**: You'll find detailed information about our project history, features, and roadmap on our documentation page.
* **Supporting Diversity and Inclusion**: We strive to make sure everyone can participate in shaping Envoy Gateway. Join us today!

## Customers

Our satisfied customers are using Envoy Gateway to manage their traffic and improve the stability of their applications:

### Airspace Link
Route all public APIs to Kubernetes clusters, leveraging flexible manifest files for automatic provisioning.

### Canva
Dynamically add route descriptions and integrate with internal systems, ensuring user-friendly experiences.

### Cortex
Manage API traffic as a modern Kubernetes ingress replacement, allowing developers to focus on what matters most – code.

### Docker
Routing internal traffic for Docker Hub makes use of Envoy Gateway's best practices.

### Kubermatic
Offering multi-cluster L7 Load Balancing using GatewayAPI and leveraging Envoy Gateway capabilities.

### nemlig.com
Route internal and externally exposed services, enforcing security policies with JWT authentication on our platform.

### SenseTime
Routing most traffic to SenseCore for a seamless experience.

[Other Customers listed]

## Careers/Jobs

Ready to join the team that's building Envoy Gateway? We're committed to fostering an inclusive workplace culture where everyone can succeed:

* **Join Our Team**: Browse open positions and opportunities on our GitHub page or Slack community channel.
* **Get Involved**: Join discussions and stay updated on the latest developments.
* **Be Heard**: Share your ideas - we want to hear from you!

### Benefits of Working with Envoy Gateway

* Contribute to a rapidly evolving, open-source project
* Make an impact on a widely-adpired and adopted technology in the industry
* Work with smart talent who is passionate about innovation and collaboration

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [23]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [46]:
stream_brochure("Envoy Gateway", "https://gateway.envoyproxy.io/")

Found links: {'links': [{'type': 'about page', 'url': 'https://gateway.envoyproxy.io'}, {'type': 'contributions page', 'url': 'https://gateway.envoyproxy.io/contributions'}]}


**Envoy Gateway Brochure**
==========================

**About Us**
------------

Envoy Gateway is an open-source edge gateway project built by the community with the goal of making it easier than ever to leverage Envoy Proxy as a Kubernetes Gateway. Our mission is to provide a simple, secure, and scalable way to manage application and API traffic for enterprises.

**Our Story**
-------------

We were born out of a need to simplify the complexities of deploying, using, and managing Envoy Proxy as an API gateway. We gathered thousands of individuals from around the world who share this vision and came together to create a solution that's easy to adopt, use, and manage. Today, we're proud to be one of the fastest-growing open-source projects in the industry.

**Capabilities**
----------------

### Easy Adoption

* Deploy as a Standalone or Kubernetes-based API Gateway
* Implements the Kubernetes Gateway API for seamless integration with Envoy Proxy

### Security Controls Made Easy

* Enforce security controls, including mTLS, JWT-based access control, OIDC integration, and more
* Use our built-in Security Policy to simplify your security setup

### Traffic Management

* Advanced traffic management features, including rate limiting, retry policies, circuit breaking, timeouts, failover, and more

### Observability

* Rich set of observability features, including metrics, access logging, distributed tracing, and more

**Customer Success Stories**
---------------------------

We're not just another open-source project - we have real-world customers who are using us to transform their applications. Here's what some of them have to say:

* Canva: "Envoy Gateway has become an integral part of our routing and gateway setup." (Routing user uploads, internal systems)
* Tetrate: "TEG (Enterprise Gateway) uses Envoy Gateway to deliver applications securely and authenticate user traffic." (Enterprise Gateway for end-users)
* Airspace Link: "We route all public APIs to Kubernetes clusters using Envoy Gateway. It's been a game-changer for our traffic management." (Route API traffic)

**Join Our Community**
---------------------

Want to be part of our journey? Join us on:

* Slack
* GitHub (contributing and discussing features)
* Attend community meetings

### Contributing

Got feature ideas or feedback? Raise them by opening a GitHub issue!

We're built by the community, for the community. Let's build something amazing together.

**Get Started**
---------------

Deploy Envoy Gateway today and start simplifying your application and API traffic management.

[GitHub Link](https://github.com/envoygateway)
[Documentation Portal](https://envoygateway.readthedocs.io/)

Join us on this exciting journey!

In [45]:
# Try changing the system prompt to the humorous version when you make the Brochure for Envoy Gateway:

stream_brochure("Envoy Gateway", "https://gateway.envoyproxy.io/")

Found links: {'links': [{'type': 'about page', 'url': 'https://gateway.envoyproxy.io/docs/about/'}, {'type': 'contribution page', 'url': 'https://gateway.envoyproxy.io/contributions/'}]}


**Envoy Gateway Brochure**
==========================

### Introduction

Envoy Gateway is an open-source gateway that makes it easy to adopt, use, and manage Envoy Proxy as a Kubernetes API Gateway. It provides a simple and reliable way to deploy and manage APIs, without requiring extensive knowledge of Envoy Proxy.

### Overview of Envoy Gateway

*   Built together by the community using the Kubernetes Gateway API
*   Extends Envoy features for easy adoption
*   Supports advanced traffic management and control features
*   Provides rich observability features including metrics, access logging, distributed tracing, and more

### Key Capabilities

*   **Manage Traffic**: Envoy Gateway supports rate limiting, retry policies, circuit breaking, timeouts, failover, and more to ensure reliable API operations.
*   **Security Controls Made Easy**: Leverage the Envoy Gateway Security Policy to enforce security controls including mTLS, JWT based access control, OIDC integration, API Key based authorization, and more.
*   **Observability**: Get insights into your APIs with metrics, access logging, distributed tracing, and more.

### Adopters

Envoy Gateway is used by some of the world's leading Cloud, Tech, FinTech and DevOps companies including:

*   Canva
*   Tetrate Inc.
*   Airspace Link Ltd
*   Teleport Inc.
*   Tencent Cloud
*   QuantCo
*   Titan LLC
*   CoactiveAI

### Join the Community

Interested in using Envoy Gateway or contributing to it? Join our community on:

*   Slack: `#gateway-dev` and `#gateway-users`
*   GitHub: Open an issue or pull request today!

**What You Can Expect from Envoy Gateway**

Make API management a breeze with Envoy Gateway. Get started with our simple documentation and tutorials.

Join our community of developers, administrators, and users to share knowledge, ask questions, and get up-to-date information on the latest Envoy Gateway releases.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>